In [ ]:
#import dependencies 
import requests
import json
import pandas as pd

In [ ]:
#connect to the API (rapid api)
url = "https://live-golf-data.p.rapidapi.com/leaderboard"

querystring = {"tournId":"014","year":"2022"}
headers = {
	"X-RapidAPI-Host": "live-golf-data.p.rapidapi.com",
	"X-RapidAPI-Key": "5f629a65d7msh1010ca075b711c3p1ace3cjsndaf17672e132"
}

response = requests.get(url, headers=headers, params=querystring).json()
#find the main dictionary key, in this case it's leaderboardrows
response['leaderboardRows']

In [ ]:
#pull data on another dictionary key to find the information on the specific round
response['leaderboardRows'][0]['rounds'][0]

In [ ]:
cut_Score = int(response['cutLines'][0]['cutScore'])

In [ ]:
current_round = response['roundId']['$numberInt']

In [ ]:
df = pd.DataFrame(columns=["position","full_name","total", "current_round","currentRoundScore","cut_score","player_id","status","strokes","roundId","round_score"])

In [ ]:
for data in response['leaderboardRows']:
    first_name = data['firstName']
    last_name = data['lastName']
    position = data['position']
    player_id = float(data['playerId'])
    total = data['total']
    currentroundscore = data['currentRoundScore']
    status = data['status']
    full_name = first_name +' '+ last_name
    #comment out below before 1st round completes
    for rounds in data['rounds']:
        round_Id = rounds['roundId']['$numberInt']
        round_score = rounds['scoreToPar']
        strokes = rounds['strokes']['$numberInt']
        
        df= df.append({'position':position, 'full_name':full_name,'player_id':player_id, 'total':total, 'currentRoundScore':currentroundscore, 'cut_score':cut_Score, 'current_round':current_round, 'status':status,'roundId':round_Id, 'round_score':round_score, 'strokes':strokes}, ignore_index=True)
df['total']=df['total'].replace('E',0)
df['round_score']= df['round_score'].replace('E',0)
#df['round_score']= pd.to_numeric(df['round_score'])




In [ ]:
df['round_score']= pd.to_numeric(df['round_score'])
df['round_score']

In [ ]:
df['g']=df.groupby(['full_name','player_id']).cumcount().add(1)
df3=(pd.pivot_table(df,index=['full_name','player_id'],
                    values='round_score',
                    columns=['roundId'],
                    aggfunc='max'
                ))
df3

In [ ]:
df.to_csv('leaderboard.csv', encoding='utf-8', index=False)